## Robô com os modelos de predição utilizados para os alunos abaixo:
 **📚 MBA - Business Analytics & Big Data - T9 - FGV Paulista**
 
 ↪Bruno Pioli
 
 ↪Guilherme Alves
 
 ↪João Pedro Dannemann
 
 ↪Ricardo Santos


# MODELO 1> ANÁLISE LSTM BIDIRECIONAL SIMPLIFICADA (1 VARIÁVEL, 1 VALOR)

No primeiro modelo, usaremos a mesma lógica do modelo do professor para checar os preços a cada minuto.
- Se a expectativa for de subida (+0,02%) e possuirmos tem dinheiro em caixa, o modelo irá comprar o equivalente a USD 500
- Se a expectativa for de queda (-0,02%) e possuirmos moeda em caixa, o modelo irá vender o equivalente a USD 500

In [ ]:
import pickle
import time
from robo_BGJR import LSTMunivar,split_sequence,api_get, api_post, get_result, compute_quantity, how_much_i_have
from robo_BGJR import LSTMvetor, split_sequenceVet
import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import math

In [ ]:
# Carrega o modelo criado no notebook anterior
token = '039ddfe6fc4ef0f'
ticker=['BTCUSDT','DOGEUSDT','ETHUSDT']
valor_compra_venda = 500
x=1 # a variável x será usada para contar a quantidade de rodadas feitas

In [ ]:
# Loop que checa o preço a cada minuto e toma as decisões de compra e venda.
while True:
# Loop Interno 1: Para obter a tabela das ultimas 500 cotações de cada moeda e:
# Cria 2 Arrays: 1. Ultimo Valor de cada moeda 2: Próximo valor estimado
        
    proxvalor=[]
    ultimovalor=[]
    tendencia=[]
    qtdade=[]
    for i in range(3):
        df = api_post('cripto_quotation', {'token': token, 'ticker': ticker[i]})
        prov=float(LSTMunivar(df['close'],3)) 
        proxvalor.append(prov)
        prov2 = df.iloc[[np.argmax(df['datetime'])]]['close']
        ultimovalor.append(float(prov2))    

#Loop interno 2:
    #utilizando a mesma lógica apresentada pelo robô do professor, o loop define a quantidade de cripto à comprar de cada uma das 3 moedas:
    
    for i in range(3):
        prov = compute_quantity(coin_value = ultimovalor[i], invest_value = valor_compra_venda, significant_digits = 4)
        qtdade.append(prov)    
    #a segunda parte deste loop é usada para definir a tendência de cada moeda:
        #Ela é definida como sendo o % de variação da previsão (proxvalor) para o ultimo fechamento (ultimovalor):
        prov =(float(proxvalor[i])-ultimovalor[i])/ultimovalor[i]
        tendencia.append(prov)
    # Print do datetime atual
    print('-------------------')
    print('Rodada nº',x)
    now = pd.to_datetime('now')
    print(f'{now}')
#loop interno 3: verifica a regra de tendência e realiza a operação de compra/venda para cada uma das 3 moedas:
    #Caso as condições de compra e venda não forem atingidas, o processo não realiza uma transação
    for i in range(3):
        if tendencia[i] > 0.0002:
            # Modelo detectou uma tendência positiva da moeda i
            print(str(ticker[i]),": Está com tendência positiva de ",{str(tendencia[i])})
        
            # Verifica quanto dinheiro tem em caixa
            qtdade_money = how_much_i_have('money', token)
        
            if qtdade_money>0:
                # Se tem dinheiro, tenta comprar o equivalente à quantidade ou o máximo que o dinheiro permitir
                max_qtdade = compute_quantity(coin_value = ultimovalor[i], invest_value = qtdade_money, significant_digits = 4)
                qtdadeuso = min(qtdade[i], max_qtdade)
            
            # Realizando a compra
            print(f'Comprando {str(qtdadeuso)} {ticker[i]}')
            api_post('buy', payload = {'token': token, 'ticker': ticker[i], 'quantity': qtdadeuso})
        
        elif tendencia[i] < -0.0002:
            # Modelo detectou uma tendência negativa da moeda i
            print(str(ticker[i]),": Está com tendência negativa de ",{str(tendencia[i])})
        
            # Verifica quanto tem da moeda em caixa
            qtdade_coin = how_much_i_have(ticker[i], token)
        
            if qtdade_coin>0:
                # Se tenho a moeda, vou vender!
                qtdadeuso = min(qtdade_coin, qtdade[i])
                print(f'Vendendo {str(qtdadeuso)} {ticker[i]}')
                api_post('sell', payload = {'token': token, 'ticker': ticker[i], 'quantity':qtdadeuso})
        else:
            # Não faz nenhuma ação, espera o próximo loop
            print(str(ticker[i]),": Está com tendência neutra de ",{str(tendencia[i])},". Nenhuma ação realizada")
    x=x+1
    # Print do status após cada iteração
    print(api_post('status', payload = {'token': token}))
    time.sleep(60)

In [ ]:
# Para não ser impactado por variações da Cripto (positivas ou negativas) quando o robô não está ativado, o código abaixo vende toda a moeda disponível
for i in range(3):
    qtdade_coin = how_much_i_have(ticker[i], token)
    print(f'Vendendo {str(qtdade_coin)} {ticker[i]}')
    api_post('sell', payload = {'token': token, 'ticker': ticker[i], 'quantity':qtdade_coin})
    print(api_post('status', payload = {'token': token}))